# Indicators creation


In [5]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import Markdown, display
from sklearn.tree import DecisionTreeClassifier,export_text
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import kruskal

In [6]:
samples = pd.read_excel('../data/samples.xlsx', sheet_name=None)

train = samples[list(samples.keys())[0]]
test = samples[list(samples.keys())[1]]
oot = samples[list(samples.keys())[2]]

In [7]:
epsilon = 1e-6

# Calcul des nouvelles variables
train['ratio_debiteurs_3mois'] = (train['CRTAD_AG_NBJDE_BA'] + train['CRTAD_AG_NBJDE_BC']) / train['CRTAD_AG_NBECR_A']
train['ratio_ecritures'] = train['CRTAD_AG_NBECR_A'] / (train['ANCIENNETE'] + epsilon)
train['ratio_ecritures'] = train['CRTAD_AG_NBECR_A'] / (train['ANCIENNETE'] + epsilon)
train['ratio_age_anciennete'] = train['AGEPRS'] / (train['ANCIENNETE'] + epsilon)
train['ratio_mvt_aff_soldes'] = (train['CRTAD_AG_MVTAFF_IND_0010'] / train['CRTAD_AG_SOLDE_T']).fillna(-1)
train['ratio_jours_debiteurs'] = train['CRTOC_IND_0015'] / (train['ANCIENNETE'] + epsilon)
train['variation_soldes_moyens'] = (train['CRTAD_AG_SOLDE_T'] - train['CRTAD_AG_SOLDE_T1']) / train['CRTAD_AG_SOLDE_T1']
train['var_jours_debiteurs_historique'] = train['CRTAD_AG_NBJDE_BC'] - train['CRTAD_AG_NBJDE_BA']
train['ecritures_mvt_aff_ratio_diff'] = (
    (train['CRTAD_IND_0042'] / train['CRTAD_AG_MVTAFF_IND_0010']) -
    (train['CRTAD_AG_NBECR_A'] / train['CRTAD_AG_MVTAFF_IND_0010']))
train['var_soldes_moyens'] = train['CRTAD_AG_SOLDE_T1'] - train['CRTAD_AG_SOLDE_T2']
train['var_soldes_long_terme'] = train['CRTAD_AG_SOLDE_T2'] - train['CRTAD_AG_SOLDE_T3']
train['ecart_sectoriel_solde'] = train['CRTAD_AG_SOLDE_T'] - train.groupby('CODNAF2')['CRTAD_AG_SOLDE_T'].transform('median')
train['mouvement_affaire_12mois'] = train['CRTOC_AG_MVTAFF_IND_0010'] / 12
train['log_solde_moyen'] = np.log(train['CRTAD_IND_0038'].replace(0, np.nan))
train['anciennete_diff_age'] = train['ANCIENNETE'] - train['AGEPRS'] * 12
train['var_nb_ecritures'] = train['CRTAD_AG_NBECR_A'] - train['CRTAD_AG_NBECR_B']
train['var_glissante_jours_debiteurs'] = train['CRTAD_AG_NBJDE_BC'].rolling(window=3).mean()
train['solde_moyen_pondere'] = train['CRTOC_SLD_MOY'] * (train['CRTOC_AG_NBJDEB_IND_0015'] / 365)
train['moyenne_jours_debiteurs'] = (train['CRTOC_AG_NBJDEB_IND_0015'] + train['CRTOC_AG_NBJDEB_2T_IND_0015']) / 2
train['dominance_crediteur'] = (train['CRTOC_AG_NBJCRE_2T_IND_0009'] > train['CRTOC_AG_LIGDEB_2T_IND_0015']).astype(int)



test['ratio_debiteurs_3mois'] = (test['CRTAD_AG_NBJDE_BA'] + test['CRTAD_AG_NBJDE_BC']) / test['CRTAD_AG_NBECR_A']
test['ratio_ecritures'] = test['CRTAD_AG_NBECR_A'] / (test['ANCIENNETE'] + epsilon)
test['ratio_ecritures'] = test['CRTAD_AG_NBECR_A'] / (test['ANCIENNETE'] + epsilon)
test['ratio_age_anciennete'] = test['AGEPRS'] / (test['ANCIENNETE'] + epsilon)
test['ratio_mvt_aff_soldes'] = (test['CRTAD_AG_MVTAFF_IND_0010'] / test['CRTAD_AG_SOLDE_T']).fillna(-1)
test['ratio_jours_debiteurs'] = test['CRTOC_IND_0015'] / (test['ANCIENNETE'] + epsilon)
test['variation_soldes_moyens'] = (test['CRTAD_AG_SOLDE_T'] - test['CRTAD_AG_SOLDE_T1']) / test['CRTAD_AG_SOLDE_T1']
test['var_jours_debiteurs_historique'] = test['CRTAD_AG_NBJDE_BC'] - test['CRTAD_AG_NBJDE_BA']
test['ecritures_mvt_aff_ratio_diff'] = (
    (test['CRTAD_IND_0042'] / test['CRTAD_AG_MVTAFF_IND_0010']) -
    (test['CRTAD_AG_NBECR_A'] / test['CRTAD_AG_MVTAFF_IND_0010']))
test['var_soldes_moyens'] = test['CRTAD_AG_SOLDE_T1'] - test['CRTAD_AG_SOLDE_T2']
test['var_soldes_long_terme'] = test['CRTAD_AG_SOLDE_T2'] - test['CRTAD_AG_SOLDE_T3']
test['ecart_sectoriel_solde'] = test['CRTAD_AG_SOLDE_T'] - test.groupby('CODNAF2')['CRTAD_AG_SOLDE_T'].transform('median')
test['mouvement_affaire_12mois'] = test['CRTOC_AG_MVTAFF_IND_0010'] / 12
test['log_solde_moyen'] = np.log(test['CRTAD_IND_0038'].replace(0, np.nan))
test['anciennete_diff_age'] = test['ANCIENNETE'] - test['AGEPRS'] * 12
test['var_nb_ecritures'] = test['CRTAD_AG_NBECR_A'] - test['CRTAD_AG_NBECR_B']
test['var_glissante_jours_debiteurs'] = test['CRTAD_AG_NBJDE_BC'].rolling(window=3).mean()
test['solde_moyen_pondere'] = test['CRTOC_SLD_MOY'] * (test['CRTOC_AG_NBJDEB_IND_0015'] / 365)
test['moyenne_jours_debiteurs'] = (test['CRTOC_AG_NBJDEB_IND_0015'] + test['CRTOC_AG_NBJDEB_2T_IND_0015']) / 2
test['dominance_crediteur'] = (test['CRTOC_AG_NBJCRE_2T_IND_0009'] > test['CRTOC_AG_LIGDEB_2T_IND_0015']).astype(int)

oot['ratio_debiteurs_3mois'] = (oot['CRTAD_AG_NBJDE_BA'] + oot['CRTAD_AG_NBJDE_BC']) / oot['CRTAD_AG_NBECR_A']
oot['ratio_ecritures'] = oot['CRTAD_AG_NBECR_A'] / (oot['ANCIENNETE'] + epsilon)
oot['ratio_ecritures'] = oot['CRTAD_AG_NBECR_A'] / (oot['ANCIENNETE'] + epsilon)
oot['ratio_age_anciennete'] = oot['AGEPRS'] / (oot['ANCIENNETE'] + epsilon)
oot['ratio_mvt_aff_soldes'] = (oot['CRTAD_AG_MVTAFF_IND_0010'] / oot['CRTAD_AG_SOLDE_T']).fillna(-1)
oot['ratio_jours_debiteurs'] = oot['CRTOC_IND_0015'] / (oot['ANCIENNETE'] + epsilon)
oot['variation_soldes_moyens'] = (oot['CRTAD_AG_SOLDE_T'] - oot['CRTAD_AG_SOLDE_T1']) / oot['CRTAD_AG_SOLDE_T1']
oot['var_jours_debiteurs_historique'] = oot['CRTAD_AG_NBJDE_BC'] - oot['CRTAD_AG_NBJDE_BA']
oot['ecritures_mvt_aff_ratio_diff'] = (
    (oot['CRTAD_IND_0042'] / oot['CRTAD_AG_MVTAFF_IND_0010']) -
    (oot['CRTAD_AG_NBECR_A'] / oot['CRTAD_AG_MVTAFF_IND_0010']))
oot['var_soldes_moyens'] = oot['CRTAD_AG_SOLDE_T1'] - oot['CRTAD_AG_SOLDE_T2']
oot['var_soldes_long_terme'] = oot['CRTAD_AG_SOLDE_T2'] - oot['CRTAD_AG_SOLDE_T3']
oot['ecart_sectoriel_solde'] = oot['CRTAD_AG_SOLDE_T'] - oot.groupby('CODNAF2')['CRTAD_AG_SOLDE_T'].transform('median')
oot['mouvement_affaire_12mois'] = oot['CRTOC_AG_MVTAFF_IND_0010'] / 12
oot['log_solde_moyen'] = np.log(oot['CRTAD_IND_0038'].replace(0, np.nan))
oot['anciennete_diff_age'] = oot['ANCIENNETE'] - oot['AGEPRS'] * 12
oot['var_nb_ecritures'] = oot['CRTAD_AG_NBECR_A'] - oot['CRTAD_AG_NBECR_B']
oot['var_glissante_jours_debiteurs'] = oot['CRTAD_AG_NBJDE_BC'].rolling(window=3).mean()
oot['solde_moyen_pondere'] = oot['CRTOC_SLD_MOY'] * (oot['CRTOC_AG_NBJDEB_IND_0015'] / 365)
oot['moyenne_jours_debiteurs'] = (oot['CRTOC_AG_NBJDEB_IND_0015'] + oot['CRTOC_AG_NBJDEB_2T_IND_0015']) / 2
oot['dominance_crediteur'] = (oot['CRTOC_AG_NBJCRE_2T_IND_0009'] > oot['CRTOC_AG_LIGDEB_2T_IND_0015']).astype(int)

In [8]:
with pd.ExcelWriter('../data/samples.xlsx') as writer:
    train.to_excel(writer, sheet_name='train', index=False)
    test.to_excel(writer, sheet_name='test', index=False)
    oot.to_excel(writer, sheet_name='oot', index=False)